In [ ]:
!pip install spotipy
!pip install beautifulsoup4 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 7.0 MB/s eta 0:00:00


Cấp quyền truy cập vào thông tin tài khoản

In [ ]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import csv


In [ ]:
# Thiết lập OAuth 2.0 với các thông tin đã lấy từ Spotify Developer Dashboard
CLIENT_ID = '0477998cd7304c888f06a6ced8eeeac3'
CLIENT_SECRET = 'dc5fa07cdc4b4f27b3baa4e08a90e674'
REDIRECT_URI = 'http://localhost:8888/callback'
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
))

# ZING_API_BASE_URL = "https://zingmp3.vn/api/v2/song/get/lyrics"

def get_song_in_playlist(playlist_id):

    results = sp.playlist_items(playlist_id)
    songs = []

    while results:
        # Duyệt qua từng bài hát trong trang kết quả
        for item in results['items']:
            track = item['track']
            if track:  # Kiểm tra xem bài hát có hợp lệ
                song_info = {
                    'song_id'  : track['id'],
                    'song_name': track['name'],  # Tên bài hát
                    'artist_name': ', '.join(artist['name'] for artist in track['artists']),  # Tên nghệ sĩ
                    'song_URL' : track['external_urls']['spotify'],  # Link bài hát trên Spotify
                }
                songs.append(song_info)

        # Kiểm tra và lấy trang tiếp theo nếu có
        results = sp.next(results) if results['next'] else None

    return songs


def get_zing_id(song_name, artist_name):
    """
    Tìm kiếm bài hát trên Zing MP3 và lấy ID bài hát.

    Args:
        song_name (str): Tên bài hát.
        artist_name (str): Tên nghệ sĩ.

    Returns:
        str: ID bài hát trên Zing MP3 hoặc thông báo lỗi.
    """
    try:
        params = {
            "q": f"{song_name} {artist_name}",
            "type": "song",
        }
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Referer": "https://zingmp3.vn/",
        }

        response = requests.get(ZING_SEARCH_URL, params=params, headers=headers, timeout=10)

        # Debug thông tin phản hồi
        print("Search Response URL:", response.url)
        print("Search Response Status Code:", response.status_code)
        print("Search Response Text:", response.text[:500])  # Giới hạn in

        # Kiểm tra phản hồi
        if 'application/json' not in response.headers.get('Content-Type', ''):
            return None, f"Error: Non-JSON response received. Content: {response.text[:100]}"

        data = response.json()

        # Kiểm tra kết quả tìm kiếm
        if "data" in data and "songs" in data["data"] and len(data["data"]["songs"]) > 0:
            song = data["data"]["songs"][0]  # Lấy bài hát đầu tiên trong danh sách
            return song["id"], None
        else:
            return None, "No song found on Zing MP3."

    except Exception as e:
        return None, f"An error occurred: {e}"

def save_to_csv(filename, data, fieldnames):
    """
    Lưu danh sách bài hát và lời bài hát vào file CSV.

    Args:
        filename (str): Tên file CSV.
        data (list): Danh sách các bài hát và thông tin.
        fieldnames (list): Danh sách tên cột cho file CSV.
    """
    with open(filename, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)




In [ ]:
Nhac_vui_id = '4bh1RU0v3SNqu84sgqUzhg'
Nhac_buon_id = '6pFxemaH9g9J9P8nxoRJBs'

# Lấy danh sách bài hát từ playlist
Nhac_vui_songs = get_song_in_playlist(Nhac_vui_id)
Nhac_buon_songs = get_song_in_playlist(Nhac_buon_id)

# Lưu danh sách bài hát và lời bài hát vào CSV
fieldnames_songs = ['song_id', 'song_name', 'artist_name', 'song_URL']

save_to_csv('Nhac_vui_songs.csv', Nhac_vui_songs, fieldnames_songs)
save_to_csv('Nhac_buon_songs.csv', Nhac_buon_songs, fieldnames_songs)


In [ ]:
print(Nhac_vui_songs[3])

{'song_id': '3bdTmaKivmPwheik5Y409t', 'song_name': 'Váy Hoa Nhí', 'artist_name': 'Hoàng Minh Châu', 'song_URL': 'https://open.spotify.com/track/3bdTmaKivmPwheik5Y409t'}
